In [1]:
!pip install git+https://github.com/ibm-granite-community/utils \
    "langchain_community<0.3.0" \
    replicate

  Cloning https://github.com/ibm-granite-community/utils to /tmp/pip-req-build-e0lhdtug
  Running command git clone --filter=blob:none --quiet https://github.com/ibm-granite-community/utils /tmp/pip-req-build-e0lhdtug
  Resolved https://github.com/ibm-granite-community/utils to commit aa05c43dc5ee022083221f3db59adc2ec869d50a
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.6/48.6 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.1/397.1 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.8/311.8 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 68.7 MB

In [1]:
from ibm_granite_community.notebook_utils import get_env_var
from langchain_community.llms import Replicate

model = Replicate(
    model="ibm-granite/granite-3.3-8b-instruct",
    replicate_api_token=get_env_var('REPLICATE_API_TOKEN'),
    model_kwargs={"max_tokens":1024, "temperature":0.2},
)

REPLICATE_API_TOKEN loaded from Google Colab secret.


In [2]:
def fewshot_prompt(context, question, book_titles, examples):
    """
    Creates a few-shot prompt for the model, where the model leverages examples to improve accuracy.

    Parameters:
    - context: str, contextual information for the prompt
    - question: str, specific question or task for the model to perform
    - book_titles: list, list of book titles to include in the prompt
    - examples: list of dicts, each containing 'question', 'context', and 'output' as keys to provide examples
    Returns:
    - str, the formatted few-shot prompt
    """

    # Format the book titles
    titles = ", ".join(book_titles)

    # Format the examples for the prompt
    formatted_examples = "\n\n".join(
        f"""
        Example {i+1}:
        User Question: {example['question']}
        Context: {example['context']}
        Model Output: {example['output']}
        """
        for i, example in enumerate(examples)
    )

    # Construct the few-shot prompt
    prompt = f"""
    You are an experienced programmer with 15 years of experience writing full-stack applications.
    Your task is to generate high-quality Python code for a Jupyter Notebook using ipywidgets UI components
    based on the provided context and user question.

    Here are some examples of similar tasks you have completed successfully:
    {formatted_examples}

    Now, using these examples as a reference, generate code for the following task:

    Context: {context}
    User Question: {question}
    Include descriptions of elements from the book titles: {titles}
    Ensure that:
    - The code is well-structured and uses consistent styling, coloring, and formatting for all UI elements.
    - Output only the Python code.
    """
    return prompt


In [3]:
# Function to get answer using few-shot prompting
def get_answer_using_fewshot(context, question, book_titles, examples):
    """
    Generates the response from the model based on a few-shot prompt.

    Parameters:
    - context: str, contextual information for the prompt
    - question: str, specific question for the model to answer
    - book_titles: list, list of book titles to include in the prompt

    Returns:
    - str, the generated result from the model
    """
    prompt = fewshot_prompt(context, question, book_titles, examples)
    result = model.invoke(prompt)

    return result

In [4]:
examples = [
{
    "question": "Add a styled header for my bookstore landing page",
    "context": "Gradient Background and Font Styling for Header",
    "output": """
import ipywidgets as widgets
from IPython.display import display

# Create a styled header
header = widgets.HTML(value="
    <div style='background: linear-gradient(to right, #6a11cb, #2575fc); padding: 20px; border-radius: 8px;'>
        <h1 style='color: white; text-align: center; font-family: Arial, sans-serif;'>"Welcome to Reader's Verse"</h1>
    </div>
")

display(header)
"""
},
{
    "question": "Enhance my bookstore catalog with hover effects",
    "context": "Interactive Book Tiles with Shadows and Hover Animation",
    "output": """
import ipywidgets as widgets
from IPython.display import display

# Create book tiles with hover effects
book_tiles = [
    widgets.HTML(value="
        <div style='background-color: #ffffff; padding: 20px; margin: 15px; border-radius: 8px; transition: transform 0.2s; box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);'
             onmouseover="this.style.transform='scale(1.05)';"
             onmouseout="this.style.transform='scale(1.0)';">
            <h2 style='color: #444;'>The Great Gatsby</h2>
            <p style='color: #555;'><b>Author:</b> F. Scott Fitzgerald</p>
            <p style='color: #555;'><b>Price:</b> $10.00</p>
        </div>
    "),
    widgets.HTML(value="
        <div style='background-color: #ffffff; padding: 20px; margin: 15px; border-radius: 8px; transition: transform 0.2s; box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);'
             onmouseover="this.style.transform='scale(1.05)';"
             onmouseout="this.style.transform='scale(1.0)';">
            <h2 style='color: #444;'>Pride and Prejudice</h2>
            <p style='color: #555;'><b>Author:</b> Jane Austen</p>
            <p style='color: #555;'><b>Price:</b> $15.00</p>
        </div>
    "),
]

container = widgets.VBox(book_tiles)
display(container)
"""
},
{
    "question": "Design and develop an online bookstore UI components with minimalistic theme.",
    "context": "Create the landing page for users visiting my bookstore. The landing page should display a header `Reader's Online Store`, a welcome message `Welcome to Reader's Verse` along with a catalog of books titles as title. Vertically align the sections.",
    "output": """
import ipywidgets as widgets
from IPython.display import display
# Create a header
header = widgets.HTML(value=\"\"\"
<div style='background-color: #333; color: white; padding: 20px; border-radius: 8px;'>
    <h1 style='text-align: center; font-family: Arial, sans-serif;'>Reader's Online Store</h1>
</div>
\"\"\")
# Create a welcome message
welcome = widgets.HTML(value=\"\"\"
<div style='padding: 20px; border-radius: 8px;'>
    <h2 style='color: white; text-align: center;'>Welcome to Reader's Verse</h2>
</div>
\"\"\")
# Create book tiles
book_titles = ["The Great Gatsby", "Pride and Prejudice", "The Hobbit", "The Lord of the Rings", "Animal Farm", "Brave New World"]
book_tiles = []
for i, title in enumerate(book_titles):
    background = "#f5f5f5" if i % 2 == 0 else "#eaeaea"
    book_tiles.append(widgets.HTML(value=f\"\"\"
<div style='background-color: {background}; padding: 15px; margin: 10px; border-radius: 8px;'>
    <h2 style='color: #333;'>{title}</h2>
    <p style='color: #555;'>Author: Author Name {i+1}</p>
    <p style='color: #555;'>Price: ${(i+1)*10}.00</p>
</div>
\"\"\"))
# Create a container for the book tiles
book_container = widgets.VBox(book_tiles)
# Create a vertical box for the header, welcome message, and book container
main_container = widgets.VBox([header, welcome, book_container])
# Display the main container
display(main_container)
"""
},
{
    "question": "Refined page with Hover Effects for Enhanced User Interaction",
    "context": "Enhance the book tiles in the catalog with hover effects that change the background color, add a shadow, and slightly scale the tiles on hover. Output should also retain the Header and Welcome message aswell in the page",
    "output": """
import ipywidgets as widgets
from IPython.display import display
# Create a header
header = widgets.HTML(value=\"\"\"
<div style='background-color: #333; color: white; padding: 20px; border-radius: 8px;'>
    <h1 style='text-align: center; font-family: Arial, sans-serif;'>Reader's Online Store</h1>
</div>
\"\"\")
# Create a welcome message
welcome = widgets.HTML(value=\"\"\"
<div style='padding: 20px; border-radius: 8px;'>
    <h2 style='color: white; text-align: center;'>Welcome to Reader's Verse</h2>
</div>
\"\"\")
# Create book tiles with hover effects
book_titles = ['The Great Gatsby', 'Pride and Prejudice', 'The Hobbit', 'The Lord of the Rings', 'Animal Farm', 'Brave New World']
book_tiles = []
for i, title in enumerate(book_titles):
    background = "#f5f5f5" if i % 2 == 0 else "#eaeaea"
    book_tiles.append(widgets.HTML(value=f\"\"\"
<div style='background-color: {background}; padding: 15px; margin: 10px; border-radius: 8px; transition: transform 0.2s; box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);'
     onmouseover="this.style.transform='scale(1.05)';"
     onmouseout="this.style.transform='scale(1.0)';">
    <h2 style='color: #333;'>{title}</h2>
    <p style='color: #555;'>Author: Author Name {i+1}</p>
    <p style='color: #555;'>Price: ${(i+1)*10}.00</p>
</div>
\"\"\"))
# Create a container for the book tiles
book_container = widgets.VBox(book_tiles)

# Create a vertical box for the header, welcome message, and book container
main_container = widgets.VBox([header, welcome, book_container])
# Display the main container
display(main_container)
"""
}
]

In [5]:
# Example usage
context = "Design and develop an online bookstore UI components with minimalistic theme."
question = "Create the landing page for users visiting my bookstore. The landing page should display a header `Reader's Online Store`, a welcome message `Welcome to Reader's Verse` along with a catalog of books titles as title. Vertically align the sections."
book_titles = ["The Great Gatsby", "Pride and Prejudice", "The Hobbit", "The Lord of the Rings", "Animal Farm", "Brave New World"]
# Generate and display the UI code for the landing page
result = get_answer_using_fewshot(context, question, book_titles, examples)
print(f"Generated Code:\n{result}")

Generated Code:
```python
import ipywidgets as widgets
from IPython.display import display

# Create a header
header = widgets.HTML(value="""
<div style='background-color: #333; color: white; padding: 20px; border-radius: 8px;'>
    <h1 style='text-align: center; font-family: Arial, sans-serif;'>Reader's Online Store</h1>
</div>
""")

# Create a welcome message
welcome = widgets.HTML(value="""
<div style='padding: 20px; border-radius: 8px;'>
    <h2 style='color: white; text-align: center;'>Welcome to Reader's Verse</h2>
</div>
""")

# Create book tiles with descriptions
book_titles = ['The Great Gatsby', 'Pride and Prejudice', 'The Hobbit', 'The Lord of the Rings', 'Animal Farm', 'Brave New World']
book_tiles = []
for i, title in enumerate(book_titles):
    background = "#f5f5f5" if i % 2 == 0 else "#eaeaea"
    description = f"A story of the Roaring Twenties, exploring themes of decadence, idealism, resistance to change, social upheaval, and excess, made famous by F. Scott Fitzgerald

In [6]:
import ipywidgets as widgets
from IPython.display import display
# Create a header
header = widgets.HTML(value="""
<div style='background-color: #333; color: white; padding: 20px; border-radius: 8px;'>
    <h1 style='text-align: center; font-family: Arial, sans-serif;'>Reader's Online Store</h1>
</div>
""")
# Create a welcome message
welcome = widgets.HTML(value="""
<div style='padding: 20px; border-radius: 8px;'>
    <h2 style='color: white; text-align: center;'>Welcome to Reader's Verse</h2>
</div>
""")
# Create book tiles
book_titles = ["The Great Gatsby", "Pride and Prejudice", "The Hobbit", "The Lord of the Rings", "Animal Farm", "Brave New World"]
book_tiles = []
for i, title in enumerate(book_titles):
    background = "#f5f5f5" if i % 2 == 0 else "#eaeaea"
    book_tiles.append(widgets.HTML(value=f"""
<div style='background-color: {background}; padding: 15px; margin: 10px; border-radius: 8px;'>
    <h2 style='color: #333;'>{title}</h2>
    <p style='color: #555;'>Author: Author Name {i+1}</p>
    <p style='color: #555;'>Price: ${(i+1)*10}.00</p>
</div>
"""))
# Create a container for the book tiles
book_container = widgets.VBox(book_tiles)
# Create a vertical box for the header, welcome message, and book container
main_container = widgets.VBox([header, welcome, book_container])
# Display the main container
display(main_container)

In [7]:
def refine_output_with_fewshot(result, context, question, book_titles, formatted_examples):
    """
    Refines a previously generated output by applying additional user-requested changes
    using a few-shot prompt technique.

    Parameters:
    - previous_output: str, the code generated from the initial prompt.
    - context: str, the context for the refinement task.
    - question: str, the user-requested changes to be applied.
    - book_titles: list, list of book titles to include in the refined output.
    - formatted_examples: str, formatted examples of similar tasks for few-shot learning.

    Returns:
    - str, the refined Python code generated by the model.
    """

    # Format the examples for the prompt
    formatted_examples = "\n\n".join(
        f"""
        Example {i+1}:
        User Question: {example['question']}
        Context: {example['context']}
        Model Output: {example['output']}
        """
        for i, example in enumerate(examples)
    )
    prompt = f"""
    You are an experienced programmer with 15 years of experience writing full-stack applications.
    Your task is to generate high-quality Python code for a Jupyter Notebook using ipywidgets UI components
    based on the provided context and user question.

    Here are some examples of similar tasks you have completed successfully:
    {formatted_examples}

    Now, I have a Landing Page UI with basic styling created from an initial prompt. Refine the Current Page:{result} based on the additional styling and functionality changes requested in the:

    Context:
    {context}

    User Question:
    {question}
    """
    # Use the model to generate the refined code
    result = model.invoke(prompt)
    return result

In [8]:
# Example usage
context = "Refined page with Hover Effects for Enhanced User Interaction"
question = "Enhance the book tiles in the catalog with hover effects that change the background color, add a shadow, and slightly scale the tiles on hover. Output should also retain the Header and Welcome message aswell in the page"
# Generate and display the UI code for the landing page
ref_result = refine_output_with_fewshot(context, question, book_titles, examples, result)
print(f"Generated Code:\n{ref_result}")

Generated Code:
Based on your request, I will refine the existing code to include hover effects for the book tiles, changing the background color, adding a shadow, and slightly scaling the tiles on hover. The header and welcome message will remain unchanged. Here's the updated code:

```python
import ipywidgets as widgets
from IPython.display import display

# Create a header
header = widgets.HTML(value="""
<div style='background-color: #333; color: white; padding: 20px; border-radius: 8px;'>
    <h1 style='text-align: center; font-family: Arial, sans-serif;'>Reader's Online Store</h1>
</div>
""")

# Create a welcome message
welcome = widgets.HTML(value="""
<div style='padding: 20px; border-radius: 8px;'>
    <h2 style='color: white; text-align: center;'>Welcome to Reader's Verse</h2>
</div>
""")

# Create book tiles with hover effects
book_titles = ['The Great Gatsby', 'Pride and Prejudice', 'The Hobbit', 'The Lord of the Rings', 'Animal Farm', 'Brave New World']
book_tiles = []
for i

In [9]:
import ipywidgets as widgets
from IPython.display import display
# Create a header
header = widgets.HTML(value="""
<div style='background-color: #333; color: white; padding: 20px; border-radius: 8px;'>
    <h1 style='text-align: center; font-family: Arial, sans-serif;'>Reader's Online Store</h1>
</div>
""")
# Create a welcome message
welcome = widgets.HTML(value="""
<div style='padding: 20px; border-radius: 8px;'>
    <h2 style='color: white; text-align: center;'>Welcome to Reader's Verse</h2>
</div>
""")
# Create book tiles with hover effects
book_titles = ['The Great Gatsby', 'Pride and Prejudice', 'The Hobbit', 'The Lord of the Rings', 'Animal Farm', 'Brave New World']
book_tiles = []
for i, title in enumerate(book_titles):
    background = "#f5f5f5" if i % 2 == 0 else "#eaeaea"
    book_tiles.append(widgets.HTML(value=f"""
<div style='background-color: {background}; padding: 15px; margin: 10px; border-radius: 8px; transition: transform 0.2s; box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);'
     onmouseover="this.style.transform='scale(1.05)';"
     onmouseout="this.style.transform='scale(1.0)';">
    <h2 style='color: #333;'>{title}</h2>
    <p style='color: #555;'>Author: Author Name {i+1}</p>
    <p style='color: #555;'>Price: ${(i+1)*10}.00</p>
</div>
"""))
# Create a container for the book tiles
book_container = widgets.VBox(book_tiles)

# Create a vertical box for the header, welcome message, and book container
main_container = widgets.VBox([header, welcome, book_container])
# Display the main container
display(main_container)